# Tire Analysis: Temperature Distribution & Thermography

This notebook visualizes tire temperature data from infrared sensors across your tires, helping you understand tire behavior and identify setup issues.

## What You'll Find Here

- **Tire Temperature Heatmaps**: Visual representation of temperature distribution across all four tires (FL, FR, RL, RR) throughout the lap
- **Temperature vs Distance**: See how tire temperatures change through different track sections
- **Speed & G-Force Overlay**: Correlate tire temperatures with speed and combined lateral/longitudinal acceleration
- **Driver Inputs Overlay**: See throttle, brake, and steering inputs aligned with temperature data

## How to Interpret the Results

### Temperature Heatmaps
- **Ch1-Ch8**: Represent temperature sensor positions across the tire width
  - FL/RL: Ch1 = Outside (left), Ch8 = Inside (right)
  - FR/RR: Ch1 = Inside (left), Ch8 = Outside (right)
- **Hot spots (bright)**: Areas of high temperature - could indicate excessive load or slip
- **Cold spots (dark)**: Areas not being worked - potential grip left on the table
- **Even temperature gradient**: Indicates good tire usage and camber settings

### Setup Insights
- **Outside edge hot**: May need more negative camber
- **Inside edge hot**: May have too much negative camber
- **Center hot**: Could indicate over-inflation
- **Edges hot, center cold**: Could indicate under-inflation
- **Front vs Rear difference**: Balance insights for understeer/oversteer tendencies

## Using Your Own Data

To analyze your own data:

1. **Run the first cell** below to install packages and display the upload widget
2. **Click "Choose File"** to select your `.xrk` or `.xrz` file
3. **Run all remaining cells** to analyze your data

The status indicator will show which file is being used. If you don't upload a file, the sample data will be used.

## Requirements

- Tire temperature channels (`FL_Ch1`-`FL_Ch8`, `FR_Ch1`-`FR_Ch8`, `RL_Ch1`-`RL_Ch8`, `RR_Ch1`-`RR_Ch8`)
- GPS Speed channel for distance calculation
- Acceleration data (`LateralAcc`, `InlineAcc`) for G-force visualization
- Driver input channels (`BrakePress`, `PPS`, `SteerAngle`)

**Note:** This notebook works in both JupyterLite (browser) and standard JupyterLab environments.

In [1]:
# Install required packages (needed for JupyterLite, skipped in regular JupyterLab if already installed)
%pip install -q pandas plotly libxrk motorsports-data-notebook jinja2 ipywidgets

# Import helper functions
from motorsports_data_notebook.visualization import (
    format_lap_time,
    plot_tire_thermography,
    show_fig,
)
from motorsports_data_notebook.widgets import SessionPicker

# Session picker - upload your own file and select a lap to analyze
session = SessionPicker(default_file="CMD_Inferno 86_Fuji GP Sh_Generic testing_a_2248.xrz")
session.display()

Note: you may need to restart the kernel to use updated packages.


/home/runner/.cache/pypoetry/virtualenvs/motorsports-data-notebook-lSQWGACw-py3.12/lib/python3.12/site-packages/libxrk/gps.py:343: RuntimeWarning: invalid value encountered in divide
  t = np.maximum(-np.sum(SN * O, axis=1) / np.sum(SN * D, axis=1), 0)
/home/runner/.cache/pypoetry/virtualenvs/motorsports-data-notebook-lSQWGACw-py3.12/lib/python3.12/site-packages/libxrk/gps.py:356: RuntimeWarning: divide by zero encountered in divide
  t = np.maximum(-np.sum(SN * O, axis=1) / np.sum(SN * D, axis=1), 0)
/home/runner/.cache/pypoetry/virtualenvs/motorsports-data-notebook-lSQWGACw-py3.12/lib/python3.12/site-packages/libxrk/gps.py:357: RuntimeWarning: invalid value encountered in multiply
  dist = np.sum(np.square(O + t.reshape((len(t), 1)) * D), axis=1)


division: scan=0.054212, gps=0.018185, group/ch=0.065760 more


In [2]:
# Get laps as pandas DataFrame for display
laps = session.get_laps()

In [3]:
# Display lap times table
laps.style.format({"lap_time": format_lap_time})  # type: ignore[dict-item]

,num,start_time,end_time,lap_time
0,0,0,150454,2:30.454
1,1,150454,279602,2:09.148
2,2,279602,406240,2:06.638
3,3,406240,532797,2:06.557
4,4,532797,659282,2:06.485
5,5,659282,787773,2:08.491
6,6,787773,913776,2:06.003
7,7,913776,1041397,2:07.621
8,8,1041397,1168322,2:06.925
9,9,1168322,1294676,2:06.354


In [4]:
# Define channels needed for tire thermography
tire_channels = [f"{pos}_Ch{i}" for pos in ["FL", "FR", "RL", "RR"] for i in range(1, 9)]
other_channels = [
    "distance_m",
    "speed_kmh",
    "LateralAcc",
    "InlineAcc",
    "BrakePress",
    "PPS",
    "SteerAngle",
]

# Extract data for the selected lap using libxrk 0.5.0 methods
selected_lap = session.get_selected_lap()
log = session.get_log()
lap_num = int(selected_lap["num"])

# Filter to lap, select channels, and resample to distance_m timebase
channels = (
    log.filter_by_lap(lap_num)
    .select_channels(tire_channels + other_channels)
    .resample_to_channel("distance_m")
    .channels
)

In [5]:
# Tire Thermography - Selected Lap (uses on-demand interpolation)
fig = plot_tire_thermography(channels, title=f"Tire Temperatures - Lap {int(selected_lap['num'])}")
show_fig(fig)